In [ ]:
import qrcode
import requests
import math
import time
import os
import re
import sys
import html
import json
import math
import threading
import subprocess
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta, timezone
import http.cookiejar
import urllib
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA256
import binascii


# 文件保存模块
def file_save(content, file_name, folder=None):
    # 如果指定了文件夹则将文件保存到指定的文件夹中
    if folder:
        file_path = os.path.join(os.path.join(os.getcwd(), folder), file_name)
    else:
        # 如果没有指定文件夹则将文件保存在当前工作目录中
        file_path = os.path.join(os.getcwd(), file_name)
    # 保存文件
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(content)

#日志模块
def write_log(log, suffix = None, display = True, time_display = True):
    # 获取当前的具体时间
    current_time = datetime.now()
    # 格式化输出, 只保留年月日时分秒
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    # 打开文件, 并读取原有内容
    try:
        with open("log.txt", "r") as file:
            contents = file.read()
    except FileNotFoundError:
        contents = ""
    # 将新的日志内容添加在原有内容之前
    log_in = re.sub(r"\033\[[0-9;]+m", "", log)
    log_in = re.sub(r"\n", "", log_in)
    new_contents = f"{formatted_time} {log_in}\n{contents}"
    # 将新的日志内容写入文件
    file_save(new_contents, "log.txt")
    if display:
        formatted_time_mini = current_time.strftime("%H:%M:%S")
        log_print = f"{formatted_time_mini}|{log}" if time_display else f"{log}"
        log_print = f"{log_print}|{suffix}" if suffix else f"{log_print}"
        print(log_print)

#网址二维码模块
def qr_code(data):
    # 创建一个QRCode对象
    qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=1, border=0)
    # 设置二维码的数据
    qr.add_data(data)
    # 获取QR Code矩阵
    qr.make(fit=True)
    matrix = qr.make_image(fill_color="black", back_color="white").modules
    # 获取图像的宽度和高度
    width, height = len(matrix), len(matrix)
    height_double = math.ceil(height/2)
    # 转换图像为ASCII字符
    fonts = ["▀", "▄", "█", " "]
    ascii_art = ""
    for y in range(height_double):
        if (y+1)*2-1 >= height:
            for x in range(width):
                ascii_art += fonts[0] if matrix[(y+1)*2-2][x] is True else fonts[3]
        else:
            for x in range(width):
                if matrix[(y+1)*2-2][x] is True and matrix[(y+1)*2-1][x] is True:
                    ascii_art += fonts[2]
                elif matrix[(y+1)*2-2][x] is True and matrix[(y+1)*2-1][x] is False:
                    ascii_art += fonts[0]
                elif matrix[(y+1)*2-2][x] is False and matrix[(y+1)*2-1][x] is True:
                    ascii_art += fonts[1]
                else:
                    ascii_art += " "
            ascii_art += "\n"
    print(ascii_art)

# HTTP GET请求重试模块
def http_get(url, name, max_retries=10, retry_delay=6, headers_possess=False, cookies=None, data=None, cookie_jar_name=None):
    user_agent = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
    }
    err = None  # 初始化 err 变量
    response = None  # 初始化 response 变量
    # 创建一个Session对象
    session = requests.Session()
    if cookie_jar_name:
        # 创建一个MozillaCookieJar对象，指定保存文件
        cookie_jar = http.cookiejar.MozillaCookieJar(f"{cookie_jar_name}.txt")
        # 将CookieJar对象绑定到Session对象
        session.cookies = cookie_jar
    if headers_possess:
        session.headers.update(user_agent)
    if cookies:
        session.cookies.update(cookies)
    if data:
        session.params.update(data)
    for num in range(max_retries):
        try:
            response = session.get(url, timeout=5)
            response.raise_for_status()
        except Exception as e:
            if response is not None and response.status_code in {404}:
                return response
            print(
                f"{datetime.now().strftime('%H:%M:%S')}|{name}|\033[31m连接异常重试中...\033[97m{num + 1}\033[0m"
            )
            if err:
                err = f":\n{str(e)}"
            else:
                err = ""
        else:
            return response
        time.sleep(retry_delay)
    print(
        f"{datetime.now().strftime('%H:%M:%S')}|{name}|\033[31m达到最大重试次数\033[97m{max_retries}\033[0m{err}"
    )
    return response

# 申请bilibili二维码并获取token和URL模块
def request_qr_code():
    # 实际申请二维码的API请求
    response = http_get('https://passport.bilibili.com/x/passport-login/web/qrcode/generate', '申请bilibili二维码', 3, 5, True)
    data = response.json()
    return data['data']['qrcode_key'], data['data']['url']

# 扫码登录bilibili并返回状态和cookie模块
def scan_login(token):
    # 发送GET请求
    response = http_get(f'https://passport.bilibili.com/x/passport-login/web/qrcode/poll', 'bilibili扫码登录', 1, 1, True, None, {'qrcode_key': token}, 'yt_dlp_bilibili')
    data = response.json()
    cookies = response.cookies
    return data['data']['code'], cookies, data['data']['refresh_token']

# 登陆bilibili模块
def bilibili_login():
    token, url = request_qr_code()
    print(f"{datetime.now().strftime('%H:%M:%S')}|请用Bilibili App扫描登录:")
    qr_code(url)
    login_status_change = ""
    time_print = f"{datetime.now().strftime('%H:%M:%S')}|Bilibili "
    while True:
        status, cookie, refresh_token = scan_login(token)
        if status == 86101:
            continue
        elif status == 86038:
            login_status = '\033[31m二维码失效超时, 请重新运行\033[0m'
        elif status == 86090:
            login_status = '\033[32m扫描成功\033[0m'
        elif status == 0:
            login_status = '\033[32m登陆成功\033[0m'
        if login_status_change != login_status:
            if login_status == '':
                print(f"{time_print}{login_status}", end = "")
            else:
                print(f"\r{time_print}{login_status}", end = "")
        login_status_change = login_status
        if status == 86038:
            print("")
            return login_status, refresh_token
        elif status == 0:
            print("")
            return cookie, refresh_token
        time.sleep(1)

# 保存bilibili登陆成功后的cookies模块
def save_bilibili_cookies():
    bilibili_cookie, refresh_token = bilibili_login()
    if bilibili_cookie == '\033[31m二维码失效超时, 请重新运行\033[0m':
        write_log(f"Bilibili \033[31m登陆失败\033[0m")
        sys.exit(0)
    else:
        bilibili_cookie = requests.utils.dict_from_cookiejar(bilibili_cookie)
        bilibili_cookie["buvid3"] = http_get('https://api.bilibili.com/x/frontend/finger/spi', "获取buvid3", 3, 5, True).json()["data"]["b_3"]
        bilibili_cookie["refresh_token"] = refresh_token
        file_save(json.dumps(bilibili_cookie, ensure_ascii=False), "bilibili_cookies.txt")
        return bilibili_cookie

# 检查是否需要刷新
def updates_login(cookies):
    user_agent = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',}
    url = f"https://passport.bilibili.com/x/passport-login/web/cookie/info"
    response = requests.get(f"{url}", headers = user_agent, cookies = cookies)
    response = response.json()
    if response["code"] == 0:
        return response["code"], response["data"]["refresh"]
    else:
        return response["code"], None



In [4]:
try:
    with open('bilibili_cookies.txt', 'r') as file:
        cookies = file.read()
    cookies = json.loads(cookies)
    code, refresh_token = updates_login(cookies)
    print(code, refresh_token)
    if code == 0:
        if refresh_token is False:
            bilibili_cookie = cookies
#        else:

    else:
        bilibili_cookie = save_bilibili_cookies()
except Exception:
    bilibili_cookie = save_bilibili_cookies()

0 False


In [ ]:
print(http_get('https://api.bilibili.com/x/frontend/finger/spi', "获取buvid3", 3, 5, True).json()["data"]["b_3"])